# Nova Neural Network Whiteboard

This code is just me experimenting with building my own neural network, none of this has been unit tested, time or memory efficiency optimized, etc...

In [1]:
import numpy as np

In [2]:
#define our hidden layer
hidden_layer_length = 3


#define a stack of words for embedding
word_stack = {
    "hello": np.random.random((hidden_layer_length, 1))*100 - 50,
    "world": np.random.random((hidden_layer_length, 1))*100 - 50
}

word_stack

{'hello': array([[ 5.79321837],
        [19.78262095],
        [10.89987604]]),
 'world': array([[-8.99983678],
        [36.93247135],
        [37.73708553]])}

In [3]:
#Defining the "Layer" Object to build our network off of

class Layer:
    def __init__(self, W = None, a=None, b=None):
        self.W = W
        self.a = a
        self.b = b
# One activation function (would be used for everything except outputs
    @staticmethod
    def ReLU(M):
        arr = []
        for v in M.reshape(1, len(M))[0]:
            arr.append(max([v, 0]))
        return np.array(arr).reshape(len(arr), 1)

# Another activation (used for defining outputs)
    @staticmethod
    def softmax(M):
        exp_values = np.exp(M - np.max(M))  # for numerical stability
        return exp_values / np.sum(exp_values, axis=0, keepdims=True)

#feed to next layer
    def feedF(self):
        return self.ReLU(np.dot(self.W, self.a)+self.b)

#feed to outputs
    def feedO(self):
        return self.softmax(np.dot(self.W, self.a)+self.b)

Documentation on softmax and ReLU are provided below: <br>
**[ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))** <br>
**[softmax](https://en.wikipedia.org/wiki/Softmax_function)**

In [4]:
# the network itself

class NNova:
    def __init__(self, output_length):
        self.e_layer = Layer()
        self.h_layer = Layer(W = np.random.random((output_length, hidden_layer_length))*100-50,
                             b = np.random.random((output_length, 1))*100 - 50)
# queue up inputs for forward pass
    def queueInputs(self, words):
        # adds to wordstack hash map
        for word in words:
            if not word in word_stack.keys():
                print("generating new embedding")
                word_stack[word] = np.random.random((hidden_layer_length, 1))*100 - 50
        # compile into W
        for k in word_stack.keys():
            try:
                if not self.e_layer.W:
                    self.e_layer.W = word_stack[k]
            except ValueError:
                self.e_layer.W = np.dstack((self.e_layer.W, word_stack[k]))
        self.e_layer.a = [1 for w in word_stack if w in words]
        self.e_layer.b = np.random.random((hidden_layer_length, 1))*100 - 50
        return
#pass forward to output layer
    def feedForward(self):
        self.h_layer.a = self.e_layer.feedF()
        return self.h_layer.feedO()

In [5]:
#define neural network object with parameters
nn = NNova(2)

#queue up tokens
nn.queueInputs(["hello","new","world"])

generating new embedding


In [6]:
#feed forward
model_outputs = np.hstack(nn.feedForward())

# now lets say our output is...
output_vocab = ["yes", "no"]

# then in this case the model would select
answer = output_vocab[np.argmax(model_outputs)]

answer

'no'